***
Amazon [SageMaker JumpStart](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-jumpstart.html) 에 오신 것을 환영합니다 ! SageMaker Studio에서 한 번의 클릭으로 JumpStart 또는 [SageMaker JumpStart API](https://sagemaker.readthedocs.io/en/stable/overview.html#use-prebuilt-models-with-sagemaker-jumpstart) 를 사용하여 많은 기계 학습 작업을 해결할 수 있습니다, 이 데모 노트북에서는 JumpStart API를 사용하여 최첨단 Stable Diffusion 모델을 사용하여 텍스트에서 이미지를 생성하는 방법을 보여줍니다.또한 데이터세트에 맞게 모델을 미세 조정하는 방법도 보여줍니다. 

Stable Diffusion은 텍스트 프롬프트만으로 사실적인 이미지를 만들 수 있는 text-to-image 모델입니다. Diffusion 모델은 실제 이미지에 추가된 노이즈를 제거하는 방법을 학습하여 훈련합니다. 이러한 노이즈 제거 프로세스는 사실적인 이미지를 생성합니다. 또한 이러한 모델은 텍스트의 생성 프로세스를 조정하여 텍스트만으로 이미지를 생성할 수도 있습니다. 예를 들어, Stable Diffusion은 모델이 순수 노이즈 영상의 모양을 인식하는 방법을 학습한 후 형태가 입력 텍스트의 단어와 일치하면 이러한 모양에 점차 초점을 맞추는 잠재 확산 형태입니다.

Stable Diffusion과 같은 모델에서 대형 모델을 학습 및 배포하고 추론을 실행하는 것은 종종 어려운 일이며, 여기에는 메모리 부족, 페이로드 크기 제한 초과 등과 같은 문제가 포함됩니다. JumpStart는 엄격한 테스트를 거친 즉시 사용 가능한 스크립트를 제공하여 이 프로세스를 간소화합니다. 또한 권장 인스턴스 유형, 이미지 생성 프로세스를 안내하는 매개 변수 선택 방법, 신속한 엔지니어링 등을 포함하여 프로세스의 각 단계에 대한 지침을 제공합니다. 또한 코드를 직접 작성할 필요 없이 JumpStart에서 80개 이상의 Diffusion 모델 중 하나에 대해 추론을 배포하고 실행할 수 있습니다.

이 노트북의 첫 번째 부분에서는 JumpStart를 사용하여 모든 주제, 물체, 환경 또는 장면에 대해 매우 사실적이고 예술적인 이미지를 생성하는 방법을 배웁니다. 이것은 귀여운 강아지의 이미지처럼 단순할 수도 있고, 극적인 일몰 조명과 영화 같은 분위기의 긴 그림자가 있는 Greg Rutkowski 스타일의 픽서가 아름답게 장식한 아늑한 주방의 초현실적 이미지처럼 상세할 수도 있습니다. 이는 전자 상거래 비즈니스 요구에 맞는 제품을 디자인하고 카탈로그를 제작하거나 사실적인 예술 작품이나 스톡 이미지를 생성하는 데 사용할 수 있습니다.

이 노트북의 두 번째 부분에서는 JumpStart를 사용하여 데이터세트에 맞게 Stable Diffusion 모델을 미세 조정하는 방법을 알아봅니다. 이것은 아트, 로고, 맞춤형 디자인, NFT 등을 만들거나 애완동물이나 자신의 아바타의 맞춤형 AI 이미지를 생성하는 것과 같은 재미있는 것들을 만들 때 유용할 수 있습니다.


Model license: By using this model, you agree to the [CreativeML Open RAIL-M++ license](https://huggingface.co/stabilityai/stable-diffusion-2/blob/main/LICENSE-MODEL).

***

# Introduction to JumpStart - Text to Image

1. [Set Up](#1.-Set-Up)
2. [Run inference on the pre-trained model](#2.-Run-inference-on-the-pre-trained-model)
    * [Select a model](#2.1.-Select-a-Model)
    * [Retrieve JumpStart Artifacts & Deploy an Endpoint](#2.2.-Retrieve-JumpStart-Artifacts-&-Deploy-an-Endpoint)
    * [Query endpoint and parse response](#2.3.-Query-endpoint-and-parse-response)
    * [Supported Inference parameters](#2.4.-Supported-Inference-parameters)
    * [Compressed Image Output](#2.5.-Compressed-Image-Output)
    * [Prompt Engineering](#2.6.-Prompt-Engineering)
    * [Negative Prompting](#2.7.-Negative-Prompting)
    * [Clean up the endpoint](#2.8.-Clean-up-the-endpoint)

3. [Fine-tune the pre-trained model on a custom dataset](#3.-Fine-tune-the-pre-trained-model-on-a-custom-dataset)
    * [Retrieve Training Artifacts](#3.1.-Retrieve-Training-Artifacts)
    * [Set Training parameters](#3.2.-Set-Training-parameters)
    * [Start Training](#3.3.-Start-Training)
    * [Deploy and run inference on the fine-tuned model](#3.4.-Deploy-and-run-inference-on-the-fine-tuned-model)

4. [Conclusion](#4.-Conclusion)



참고: 이 노트북은 파이썬 3 (데이터 과학) 커널을 사용하는 아마존 세이지메이커 스튜디오의 ml.t3.medium 인스턴스와 conda_python3 커널을 사용하는 아마존 세이지메이커 노트북 인스턴스에서 테스트되었습니다.

참고: 사전 학습되거나 미세 조정된 모델을 배포하려면 `ml.p3.2xlarge` 또는 `ml.g4dn.2xlarge` 인스턴스 유형을 사용할 수 있습니다. 해당 region 에서 'ml.g5.2xlarge'를 사용할 수 있는 경우 해당 인스턴스 유형을 사용하여 배포하는 것이 좋습니다. 데이터세트에서 모델을 미세 조정하려면 계정에서 사용할 수 있는 `ml.g4dn.2xlarge` 인스턴스 유형이 필요합니다.

### 1. Set Up

***

노트북을 실행하기 전에 설정에 필요한 몇 가지 초기 단계가 있습니다. 이 노트북에는 ipywidgets와 최신 버전의 sagemaker가 필요합니다.
아래 셀을 실행하여 모듈을 설치합니다. 

***

In [ ]:
!pip install ipywidgets==7.0.0 --quiet
!pip install --upgrade sagemaker

#### Permissions and environment variables

***
Amazon SageMaker에서 모델을 호스팅하려면 AWS 서비스 사용을 설정하고 인증해야 합니다. 여기서는 현재 노트북과 연결된 실행 역할을 SageMaker 액세스 권한이 있는 AWS 계정 역할로 사용합니다.
아래 셀을 실행하여 환경을 설정합니다.
***

In [ ]:
import sagemaker, boto3, json
from sagemaker import get_execution_role

aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

## 2. Run inference on the pre-trained model

***
JumpStart를 사용하면 새 데이터셋에서 먼저 미세 조정하지 않고도 사전 학습된 모델을 추론할 수 있습니다.
***

### 2.1. Select a Model
***

기본 모델을 계속 사용하거나 다음 셀을 실행할 때 생성되는 드롭다운에서 다른 모델을 선택할 수 있습니다. [Sagemaker pre-trained Models](https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html#)에서 SageMaker 사전 학습된 모델의 전체 목록에 액세스할 수 있습니다.
아래 셀을 실행하여 모들을 선택합니다. 

***

In [ ]:
from ipywidgets import Dropdown
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models

# Retrieves all Text-to-Image generation models.
filter_value = "task == txt2img"
txt2img_models = list_jumpstart_models(filter=filter_value)

# display the model-ids in a dropdown to select a model for inference.
model_dropdown = Dropdown(
    options=txt2img_models,
    value="model-txt2img-stabilityai-stable-diffusion-v2-1-base",
    description="Select a model",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)
display(model_dropdown)

In [ ]:
# model_version="*" fetches the latest version of the model
model_id, model_version = model_dropdown.value, "*"

### 2.2. Retrieve JumpStart Artifacts & Deploy an Endpoint

***

JumpStart를 사용하면 새 데이터셋에서 먼저 미세 조정하지 않고도 사전 학습된 모델을 추론할 수 있습니다. 먼저 사전 학습된 모델의 경우 'deploy_image_uri'와 'model_uri'를 검색합니다. 사전 학습된 모델을 호스팅하려면 [Sagemaker.Model.Model] (https://sagemaker.readthedocs.io/en/stable/api/inference/model.html) 의 인스턴스를 만들어 배포합니다. 

### 이 작업은 대략 10분정도의 시간이 소요됩니다. 그 동안 kernel 을 kill 하지 마십시요. 

기다리는 동안 [Generate images from text with the stable diffusion model on Amazon SageMaker JumpStart](https://aws.amazon.com/blogs/machine-learning/generate-images-from-text-with-the-stable-diffusion-model-on-amazon-sagemaker-jumpstart/) 블로그를 확인하여  Stable Diffusion 모델 및 JumpStart에 대해 자세히 알아볼 수 있습니다.

아래 셀을 실행하여 모델을 호스팅합니다.

***

In [ ]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters, instance_types
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base


endpoint_name = name_from_base(f"jumpstart-example-infer-{model_id}")

# Please use ml.g5.24xlarge instance type if it is available in your region. ml.g5.24xlarge has 24GB GPU compared to 16GB in ml.p3.2xlarge and supports generation of larger and better quality images.
inference_instance_type = instance_types.retrieve_default(
    region=None,
    model_id=model_id,
    model_version=model_version,
    scope="inference"
)

# Retrieve the inference docker container uri. This is the base HuggingFace container image for the default model above.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)

# Retrieve the model uri. This includes the pre-trained model and parameters as well as the inference scripts.
# This includes all dependencies and scripts for model loading, inference handling etc..
model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="inference"
)

# To increase the maximum response size (in bytes) from the endpoint.
env = {
    "MMS_MAX_RESPONSE_SIZE": "20000000",
}

# Create the SageMaker model instance
model = Model(
    image_uri=deploy_image_uri,
    model_data=model_uri,
    role=aws_role,
    predictor_cls=Predictor,
    name=endpoint_name,
    env=env,
)



In [ ]:
# Deploy the Model. Note that we need to pass Predictor class when we deploy model through Model class,
# for being able to run inference through the sagemaker API.
model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    predictor_cls=Predictor,
    endpoint_name=endpoint_name,
)

### 2.3. Query endpoint and parse response

***

엔드포인트에 대한 입력은 json으로 변환되고 `utf-8` 형식으로 인코딩된 모든 텍스트 문자열입니다. 엔드포인트의 출력은 생성된 텍스트가 포함된 'json'입니다.
아래 셀을 실행하여 생성된 엔드포인트를 조회할 수 있는 함수를 만듭니다.

***

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def query(model_predictor, text):
    """Query the model predictor."""

    encoded_text = text.encode("utf-8")

    query_response = model_predictor.predict(
        encoded_text,
        {
            "ContentType": "application/x-text",
            "Accept": "application/json",
        },
    )
    return query_response


def parse_response(query_response):
    """Parse response and return generated image and the prompt"""

    response_dict = json.loads(query_response)
    return response_dict["generated_image"], response_dict["prompt"]


def display_img_and_prompt(img, prmpt):
    """Display hallucinated image."""
    plt.figure(figsize=(12, 12))
    plt.imshow(np.array(img))
    plt.axis("off")
    plt.title(prmpt)
    plt.show()

***
아래에는 몇 가지 예제 입력 텍스트를 입력합니다. 아무 텍스트나 입력하면 모델이 해당 텍스트에 해당하는 이미지를 예측합니다.
아래 셀을 실행하여 text 에 설정된 값에 해당하는 이미지를 볼 수 있습니다. 
***

In [ ]:
text = "cottage in impressionist style"
query_response = query(model_predictor, text)
img, prmpt = parse_response(query_response)
display_img_and_prompt(img, prmpt)

### 2.4. Supported Inference parameters

***

또한 이 모델은 추론을 수행하는 동안 많은 고급 매개변수를 지원합니다. 여기에는 다음이 포함됩니다.

* **prompt**: 이미지 생성을 안내하는 메시지. 지정해야 하며 문자열 또는 문자열 목록일 수 있습니다.
* **width**: 이미지의 너비. 지정하는 경우 이 값은 8로 나눌 수 있는 양의 정수여야 합니다.
* **height**: 이미지의 높이. 지정하는 경우 이 값은 8로 나눌 수 있는 양의 정수여야 합니다.
* **num_inference_steps**: 이미지 생성 중 발생한 노이즈 제거 단계 수입니다. 단계가 많을수록 이미지 품질이 향상됩니다. 지정하는 경우 양의 정수여야 합니다.
* **guidance_scale**: 가이드 스케일이 높을수록 프롬프트와 밀접하게 관련된 이미지가 생성되지만 이미지 품질은 떨어집니다. 지정된 경우 부동소숫점 이여야 합니다. guidance_scale<=1은 무시됩니다.
* **negative_prompt**: 이 프롬프트를 기준으로 이미지 생성을 안내합니다. 지정된 경우 문자열 또는 문자열 목록이어야 하며 guidance_scale과 함께 사용해야 합니다. guidance_scale이 비활성화되면 이 역시 비활성화됩니다. 또한 프롬프트가 문자열 목록인 경우 negative_prompt 역시 문자열 목록이어야 합니다. 
* **num_images_per_prompt**: 프롬프트당 반환되는 이미지 수입니다. 지정하는 경우 이 값은 양의 정수여야 합니다. 
* **seed**: 재현성을 위해 무작위 상태를 수정합니다. 지정하는 경우 이 값은 정수여야 합니다.

아래 셀을 실행하여 추론 파라미터를 설정합니다.

***

In [ ]:
import json

# Training data for different models had different image sizes and it is often observed that the model performs best when the generated image
# has dimensions same as the training data dimension. For dimensions not matching the default dimensions, it may result in a black image.
# Stable Diffusion v1-4 was trained on 512x512 images and Stable Diffusion v2 was trained on 768x768 images.

payload = {
    "prompt": "astronaut on a horse",
    "width": 512,
    "height": 512,
    "num_images_per_prompt": 1,
    "num_inference_steps": 50,
    "guidance_scale": 7.5,
    "seed": 1,
}


def query_endpoint_with_json_payload(model_predictor, payload, content_type, accept):
    """Query the model predictor with json payload."""

    encoded_payload = json.dumps(payload).encode("utf-8")

    query_response = model_predictor.predict(
        encoded_payload,
        {
            "ContentType": content_type,
            "Accept": accept,
        },
    )
    return query_response


def parse_response_multiple_images(query_response):
    """Parse response and return generated image and the prompt"""

    response_dict = json.loads(query_response)
    return response_dict["generated_images"], response_dict["prompt"]


query_response = query_endpoint_with_json_payload(
    model_predictor, payload, "application/json", "application/json"
)
generated_images, prompt = parse_response_multiple_images(query_response)

for img in generated_images:
    display_img_and_prompt(img, prompt)

### 2.5. Compressed Image Output

---

엔드포인트에서 보내는 위의 기본 응답 유형은 RGB 값이 있는 중첩 배열이며, 생성된 이미지 크기가 크면 응답 크기 제한에 도달할 수 있습니다. 이 문제를 해결하기 위해 각 이미지가 바이트 단위로 반환되는 JPEG 이미지로 반환되는 엔드포인트 응답도 지원합니다. 이렇게 하려면 `Accept = 'application/json;jpeg'` 로 설정하십시오.

아래 셀을 실행하여 JPEG 형태로 출력되도록 설정합니다. 

---

In [ ]:
from PIL import Image
from io import BytesIO
import base64
import json


def display_encoded_images(generated_images, title):
    """Decode the images and convert to RGB format and display

    Args:
    generated_images: are a list of jpeg images as bytes with b64 encoding.
    """

    for generated_image in generated_images:
        generated_image_decoded = BytesIO(base64.b64decode(generated_image.encode()))
        generated_image_rgb = Image.open(generated_image_decoded).convert("RGB")
        display_img_and_prompt(generated_image_rgb, title)


def compressed_output_query_and_display(payload, title):
    query_response = query_endpoint_with_json_payload(
        model_predictor, payload, "application/json", "application/json;jpeg"
    )
    generated_images, prompt = parse_response_multiple_images(query_response)

    display_encoded_images(generated_images, title)


payload = {
    "prompt": "astronaut on a horse",
    "width": 512,
    "height": 512,
    "num_images_per_prompt": 1,
    "num_inference_steps": 50,
    "guidance_scale": 7.5,
    "seed": 1,
}
compressed_output_query_and_display(payload, "generated image with compressed response type")

### 2.6. Prompt Engineering
---

좋은 프롬프트를 작성하는 것은 때때로 예술의 영역이 될 수 있습니다. 특정 프롬프트가 특정 모델에서 만족스러운 이미지를 생성할지 예측하기 어려운 경우가 많습니다. 하지만 효과가 있는 것으로 확인된 특정 템플릿도 있습니다. 프롬프트는 크게 세 가지로 나눌 수 있습니다. (i) 이미지 유형 (사진/스케치/그림 등), (ii) 설명 (주제/사물/환경/장면 등), (iii) 이미지 스타일 (사실적/예술적/예술 유형 등). 세 부분을 각각 개별적으로 변경하여 이미지의 변형을 생성할 수 있습니다. 형용사는 이미지 생성 과정에서 중요한 역할을 하는 것으로 알려져 있습니다. 또한 세부 정보를 추가하면 생성 과정에 도움이 됩니다.

사실적인 이미지를 생성하려면 a photo of, “a photograph of”, “realistic” 또는 “hyper realistic”과 같은 문구를 사용할 수 있습니다. 아티스트의 이미지를 생성하려면 “by Pablo Piccaso”, “oil painting by Rembrandt”, “landscape art by Frederic Edwin Church”, “pencil drawing by Albrecht Dürer”과 같은 문구를 사용할 수 있습니다. 여러 아티스트들을 조합할 수도 있습니다. 범주별로 예술적 이미지를 생성하려면 프롬프트에 “lion on a beach, abstract”와 같은 예술 범주를 추가할 수 있습니다. 기타 카테고리에는 “oil painting”, “pencil drawing”, “pop art”, “digital art”, “anime”, “cartoon”, “futurism”, “watercolor”, “manga” 등이 포함됩니다. 35mm 와이드 렌즈 또는 85mm 와이드 렌즈와 같은 조명 또는 카메라 렌즈와 같은 세부 정보와 프레이밍에 대한 세부 정보 (인물/풍경/클로즈업 등) 도 포함할 수 있습니다.

참고로 모델은 동일한 프롬프트가 여러 번 주어지더라도 다른 이미지를 생성합니다. 따라서 여러 이미지를 생성하고 애플리케이션에 가장 적합한 이미지를 선택할 수 있습니다.`.

아래 셀을 실행하여 prompt를 설정하고 출력을 확인하세요.

---

In [ ]:
prompts = [
    "An african woman with turban smiles at the camera, pexels contest winner, smiling young woman, face is wrapped in black scarf, a cute young woman, loosely cropped, beautiful girl, acting headshot",
    "Ancient Japanese Samurai, a person standing on a ledge in a city at night, cyberpunk art, trending on shutterstock, batman mecha, stylized cyberpunk minotaur logo, cinematic, cyberpunk",
    "Character design of a robot warrior, concept art, contest winner, diverse medical cybersuits, Football armor, triade color scheme, black shirt underneath armor, in golden armor, clothes in military armor, high resolution render, octane",
    "A croissant sitting on top of a yellow plate, a portait, trending on unsplash, sitting on a mocha-coloured table, magazine, woodfired, bakery, great composition, amber",
    "symmetry!! portrait of vanessa hudgens in the style of horizon zero dawn, machine face, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by artgerm and greg rutkowski and alphonse mucha, 8 k",
    "landscape of the beautiful city of paris rebuilt near the pacific ocean in sunny california, amazing weather, sandy beach, palm trees, splendid haussmann architecture, digital painting, highly detailed, intricate, without duplication, art by craig mullins, greg rutkwowski, concept art, matte painting, trending on artstation",
]
for prompt in prompts:
    payload = {"prompt": prompt, "width": 512, "height": 512, "seed": 1}
    compressed_output_query_and_display(payload, "generated image with detailed prompt")

### 2.7. Negative Prompting
---

Negative prompt는 Stable Diffusion 모델을 사용하여 이미지를 생성할 때 중요한 매개변수입니다. 이를 통해 이미지 생성 프로세스를 추가로 제어할 수 있으며 생성된 이미지에서 특정 개체, 색상, 스타일, 속성 등을 피하도록 모델을 지시할 수 있습니다.  
아래 셀을 실행하여 nagative prompt 를 설정하고 출력을 확인하세요.

---

In [ ]:
prompt = "emma watson as nature magic celestial, top down pose, long hair, soft pink and white transparent cloth, space, D&D, shiny background, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, artgerm, bouguereau"
payload = {"prompt": prompt, "seed": 0}
compressed_output_query_and_display(payload, "generated image with no negative prompt")


negative_prompt = "windy"
payload = {"prompt": prompt, "negative_prompt": negative_prompt, "seed": 0}
compressed_output_query_and_display(
    payload, f"generated image with negative prompt: `{negative_prompt}`"
)

---

원래 프롬프트에서 “not”, “except”, “no” 및 “not”라는 부정적인 단어를 지정하여 이러한 개념을 대부분 지정할 수 있지만, Stable Diffusion 모델은 부정적인 단어를 잘 이해하지 못하는 것으로 관찰되었습니다. 따라서 사용 사례에 맞게 이미지를 조정할 때는 네거티브 프롬프트 매개변수를 사용해야 합니다. 

---

In [ ]:
prompt = "a portrait of a man without beard"
payload = {"prompt": prompt, "seed": 0}
compressed_output_query_and_display(payload, f"prompt: `{prompt}`, negative prompt: None")

prompt, negative_prompt = "a portrait of a man", "beard"
payload = {"prompt": prompt, "negative_prompt": negative_prompt, "seed": 0}
compressed_output_query_and_display(
    payload, f"prompt: `{prompt}`, negative prompt: `{negative_prompt}`"
)

---

이미지를 생성할 때는 프롬프트부터 시작하여 점진적으로 네거티브 프롬프트를 만들어 이미지에서 원하지 않는 피사체/스타일을 제외하는 것이 좋습니다.

---


In [ ]:
prompt = "cyberpunk forest by Salvador Dali"
payload = {"prompt": prompt, "seed": 1}
compressed_output_query_and_display(payload, f"prompt: `{prompt}`, negative prompt: None")

negative_prompt = "trees, green"
payload = {"prompt": prompt, "negative_prompt": negative_prompt, "seed": 1}
compressed_output_query_and_display(
    payload, f"prompt: `{prompt}`, negative prompt: `{negative_prompt}`"
)

---
네거티브 프롬프트를 구성하는 데 유용한 키워드는 다음과 같습니다.: duplicate, blurry, Missing legs, mutation, morbid, deformed, malformed limbs, missing legs, bad anatomy, extra fingers, cloned face, too many fingers. 

---

In [ ]:
prompt = "a fantasy style portrait painting of rachel lane / alison brie / sally kellerman hybrid in the style of francois boucher oil painting unreal 5 daz. rpg portrait, extremely detailed artgerm greg rutkowski alphonse mucha"
payload = {"prompt": prompt, "seed": 1}
compressed_output_query_and_display(payload, f"No negative prompt")


negative_prompt = "duplicate"
payload = {"prompt": prompt, "negative_prompt": negative_prompt, "seed": 1}
compressed_output_query_and_display(payload, f"negative prompt: `{negative_prompt}`")

---

네거티브 프롬프트를 사용하여 프롬프트의 일부를 대체할 수도 있습니다. 예를 들어 프롬프트에서 sharp”/“focused" 을 사용하는 대신 네거티브 프롬프트에서는 “blurry”을 사용할 수 있습니다. 

네거티브 프롬프트는 특히 스테이블 디퓨전 V2((identified by model_id `model-txt2img-stabilityai-stable-diffusion-v2`, `model-txt2img-stabilityai-stable-diffusion-v2-fp16`, `model-txt2img-stabilityai-stable-diffusion-v2-1-base`)에서 중요한 것으로 관찰되었습니다. 따라서 특히 버전 2.x를 사용할 때는 네거티브 프롬프트를 사용하는 것이 좋습니다. 네거티브 프롬프트에 대한 자세한 내용은 [How to use negative prompts?](https://stable-diffusion-art.com/how-to-use-negative-prompts/) 와 [How does negative prompt work?](https://stable-diffusion-art.com/how-negative-prompt-work/)을 참조하십시오.

---

In [ ]:
#text = "a photo of a Doppler dog with a hat"
#query_response = query(finetuned_predictor, text)
#img, prmpt = parse_response(query_response)
#display_img_and_prompt(img, prmpt)

In [ ]:
prompt = "a photo of a Doppler dog with a hat"
payload = {"prompt": prompt, "seed": 0}
compressed_output_query_and_display(payload, f"prompt: `{prompt}`, negative prompt: None")

### 2.8. Clean up the endpoint

In [ ]:
# Delete the SageMaker endpoint
model_predictor.delete_model()
model_predictor.delete_endpoint()

## 3. Fine-tune the pre-trained model on a custom dataset

---

이전에는 사전 학습된 모델에서 추론을 실행하는 방법을 살펴보았습니다. 다음으로, 여러 클래스가 포함된 사용자 지정 데이터세트에 맞게 모델을 미세 조정하는 방법을 설명합니다.

모델은 모든 이미지 데이터셋에 맞게 미세 조정할 수 있습니다. 최소 5개의 훈련 이미지로도 매우 잘 작동합니다.

미세 조정 스크립트는 [dreambooth](https://dreambooth.github.io/) 의 스크립트를 기반으로 작성되었습니다. 미세 조정을 통해 반환된 모델은 추론을 위해 추가로 배포할 수 있습니다. 다음은 훈련 데이터의 형식을 지정하는 방법에 대한 지침입니다.

- **Input: ** 인스턴스 이미지, `dataset_info.json` 및 (선택 사항) 디렉토리 `class_data_dir`이 들어 있는 디렉터리입니다.
 - 이미지는 `.png` 또는 `.jpg` 또는 `.jpeg` 형식일 수 있습니다.
 - `dataset_info.json` 파일은 {'instance_prompt':<<instance_prompt>>,'class_prompt':<<class_prompt>>} 형식이어야 합니다.
 - with_prior_reservation = False인 경우, 'class_prompt'를 무시하도록 선택할 수 있습니다.
 - `class_data_dir` 디렉터리에는 클래스 이미지가 있어야 합니다. with_prior_reservation = True이고 class_data_dir이 존재하지 않거나 class_data_dir에 이미 존재하는 이미지가 충분하지 않은 경우, 추가 이미지는 class_prompt 를 사용하여 샘플링됩니다.
- **Output: ** 추론을 위해 배포할 수 있는 학습된 모델의 위치입니다.

The s3 path should look like `s3://bucket_name/input_directory/`. Note the trailing `/` is required.

s3 경로는 `s3://bucket_name/input_directory/ `과 같은 형식이어야 합니다. 참고로 마지막 `/`는 필수입니다.

다음은 훈련 데이터의 예제 형식입니다.

    input_directory
        |---instance_image_1.png
        |---instance_image_2.png
        |---instance_image_3.png
        |---instance_image_4.png
        |---instance_image_5.png
        |---dataset_info.json
        |---class_data_dir
            |---class_image_1.png
            |---class_image_2.png
            |---class_image_3.png
            |---class_image_4.png

**Prior preservation, instance prompt and class prompt:** 사전 보존은 학습하려는 동일한 클래스의 추가 이미지를 사용하는 기법입니다. 예를 들어 훈련 데이터가 특정 개(dog)의 이미지로 구성된 경우 사전 보존을 통해 일반적인 개의 클래스 이미지를 추가하는 것입니다. 특정 개를 훈련시키는 동안 여러 개의 이미지를 보여줌으로써 overfitting 을 피하려고 시도합니다. 인스턴스 프롬프트에 있는 특정 개를 나타내는 태그가 클래스 프롬프트에는 없습니다. 예를 들어, 인스턴스 프롬프트는 “a photo of a riobugger cat” 이고 클래스 프롬프트는\”a photo of a cat\”일 수 있습니다. 하이퍼 파라미터 with_prior_savetion을 True로 설정하여 사전 보존을 활성화할 수 있습니다.

우리는 고양이와 개 이미지의 기본 데이터 세트를 제공합니다. 고양이 데이터세트는 클래스 이미지가 없는 고양이 한 마리의 이미지 8개 (인스턴스 프롬프트에 해당하는 인스턴스 이미지) 로 구성됩니다.[here](https://github.com/marshmellow77/dreambooth-sm/tree/main/training-images) 에서 다운로드할 수 있습니다. 고양이 데이터세트를 사용하는 경우, 데모 노트북에서 추론을 수행하면서 “a photo of a riobugger cat”이라는 프롬프트를 눌러보세요. 강아지 데이터세트는 클래스 이미지가 없는 강아지 한 마리에 대한 12개의 이미지로 구성되어 있습니다. 강아지 데이터세트를 사용하는 경우, 데모 노트북에서 추론을 수행하면서 “a photo of a Doppler dog” 프롬프트를 사용해 보세요.

License: [MIT](https://github.com/marshmellow77/dreambooth-sm/blob/main/LICENSE).

### 3.1. Retrieve Training Artifacts

---

여기서는 훈련 도커 컨테이너, 훈련 알고리즘 소스, 사전 훈련된 기본 모델을 검색합니다. 참고로 model_version="*"는 최신 모델을 가져옵니다.

---

In [ ]:
from sagemaker import image_uris, model_uris, script_uris

# Currently, not all the stable diffusion models in jumpstart support finetuning. Thus, we manually select a model
# which supports finetuning.
train_model_id, train_model_version, train_scope = (
    "model-txt2img-stabilityai-stable-diffusion-v2-1-base",
    "*",
    "training",
)

# Tested with ml.g4dn.2xlarge (16GB GPU memory) and ml.g5.2xlarge (24GB GPU memory) instances. Other instances may work as well.
# If ml.g5.2xlarge instance type is available, please change the following instance type to speed up training.
training_instance_type = instance_types.retrieve_default(
    region=None,
    model_id=train_model_id,
    model_version=train_model_version,
    scope=train_scope
)

# Retrieve the docker image
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    model_id=train_model_id,
    model_version=train_model_version,
    image_scope=train_scope,
    instance_type=training_instance_type,
)

# Retrieve the training script. This contains all the necessary files including data processing, model training etc.
train_source_uri = script_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, script_scope=train_scope
)
# Retrieve the pre-trained model tarball to further fine-tune
train_model_uri = model_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, model_scope=train_scope
)

### 3.2. Set Training parameters

---

이제 필요한 모든 설정이 완료되었으므로 stable diffusion 모델을 훈련할 준비가 되었습니다. 먼저 [``SageMaker.estimator.estimator`](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) 객체를 만들어 보겠습니다. 이 추정기는 교육 작업을 시작합니다.


훈련을 위해 설정해야 하는 파라미터에는 두 종류가 있습니다. 첫 번째는 교육 작업의 매개변수입니다. 여기에는 다음이 포함됩니다.
- 훈련 데이터 경로. 입력 데이터가 저장되는 S3 폴더입니다.
- 출력 경로: 훈련의 결과물이 출력이 저장되는 s3 폴더입니다.
- 트레이닝 인스턴스 유형: 트레이닝을 실행할 머신의 유형을 나타냅니다. 올바른 train_image_uri를 가져오도록 위에서 교육 인스턴스 유형을 정의했습니다.
- 지표 정의: 훈련 작업을 평가하는 데 사용되는 메트릭을 정의하는 사전 목록입니다. 기본값은 SageMaker SDK에서 검색됩니다.

두 번째 파라미터 세트는 알고리즘별 트레이닝 하이퍼 파라미터입니다.

---

In [ ]:
import sagemaker.metric_definitions
# Sample training data is available in this bucket
training_data_bucket = f"jumpstart-cache-prod-{aws_region}"
training_data_prefix = "training-datasets/dogs_sd_finetuning/"

training_dataset_s3_path = f"s3://{training_data_bucket}/{training_data_prefix}"

output_bucket = sess.default_bucket()
output_prefix = "jumpstart-example-sd-training"

# Retrieve the default metric definitions to emit to CloudWatch Logs\n",
metric_definitions = sagemaker.metric_definitions.retrieve_default(
    model_id=train_model_id, model_version=train_model_version,
)

s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"

---

알고리즘별 하이퍼 파라미터의 경우 먼저 알고리즘이 디폴트 값으로 받아들이는 훈련 하이퍼 파라미터의 파이썬 딕셔너리를 가져오는 것으로 시작합니다. 그러면 이를 사용자 지정 값으로 재정의할 수 있습니다.

---

In [ ]:
from sagemaker import hyperparameters

# Retrieve the default hyper-parameters for fine-tuning the model
hyperparameters = hyperparameters.retrieve_default(
    model_id=train_model_id, model_version=train_model_version
)

# [Optional] Override default hyperparameters with custom values
hyperparameters["max_steps"] = "400"
hyperparameters["epochs"] = "5"

---

`with_prior_preservation=True`로 설정하는 경우 클래스 이미지를 생성하는 데 더 많은 메모리가 필요하므로 ml.g5.2xlarge 인스턴스 유형을 사용하십시오. 현재 ml.g4dn.2xlarge 인스턴스 유형에 대한 훈련에서는 `with_prior_preservation=True`로 설정할 때 CUDA 메모리 부족 문제가 발생했습니다.

---

### 3.3. Train with Automatic Model Tuning
---

하이퍼파라미터 튜닝이라고도 하는 Amazon SageMaker 자동 모델 튜닝은 지정한 하이퍼파라미터 알고리즘과 범위를 사용하여 데이터세트에서 많은 훈련 작업을 실행하여 모델의 최적 버전을 찾습니다. 그런 다음 하이퍼파라미터 값을 선택하여 선택한 지표로 측정했을 때 성능이 가장 우수한 모델을 만듭니다.하이퍼파라미터 튜너 객체를 사용하여 Amazon SageMaker 하이퍼파라미터 조정 API와 상호 작용할 것입니다. 여기서는 두 개의 하이퍼 파라미터 `learning_rate` 와 `max_steps` 를 조정합니다.

---

In [ ]:
from sagemaker.tuner import IntegerParameter
from sagemaker.tuner import ContinuousParameter
from sagemaker.tuner import HyperparameterTuner


use_amt = True

hyperparameter_ranges = {
    "learning_rate": ContinuousParameter(1e-7, 3e-6, "Linear"),
    "max_steps": IntegerParameter(50, 400, "Linear"),
}

### 3.4. Start Training
---


먼저 필요한 모든 에셋이 포함된 Estimator 객체를 만든 다음 교육 작업을 시작합니다. 

---

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base
from sagemaker.tuner import HyperparameterTuner

training_job_name = name_from_base(f"jumpstart-example-{train_model_id}-transfer-learning")

# Create SageMaker Estimator instance
sd_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",  # Entry-point file in source_dir and present in train_source_uri.
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    metric_definitions=metric_definitions,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    base_job_name=training_job_name,
)


if use_amt:
    # Let estimator emit fid_score metric to AMT
    sd_estimator.set_hyperparameters(compute_fid="True")
    tuner_parameters = {
        "estimator": sd_estimator,
        "metric_definitions": [{"Name": "fid_score", "Regex": "fid_score=([-+]?\\d\\.?\\d*)"}],
        "objective_metric_name": "fid_score",
        "objective_type": "Minimize",
        "hyperparameter_ranges": hyperparameter_ranges,
        "max_jobs": 1,
        "max_parallel_jobs": 1,
        "strategy": "Bayesian",
        "base_tuning_job_name": training_job_name,
    }

    tuner = HyperparameterTuner(**tuner_parameters)
    #tuner.fit({"training": training_dataset_s3_path}, logs=True)
else:
    # Launch a SageMaker Training job by passing s3 path of the training data
    #sd_estimator.fit({"training": training_dataset_s3_path}, logs=True)
    pass


위에서 `max_jobs=9` 및 `max_parallel_jobs=3`으로 설정하면 자동 모델 튜닝을 사용할 때 기본 데이터 세트에서 약 60분이 소요됩니다. 교육 인스턴스 유형에 더 많은 할당량을 사용할 수 있는 경우, 'max_parallel_jobs'를 늘려 조정 속도를 높이십시오.

### 3.5. Deploy and run inference on the fine-tuned model

---

학습된 모델은 그 자체로는 아무것도 하지 않습니다. 이제 모델을 사용하여 추론을 수행해 보겠습니다. 이 예제에서는 이미지의 바운딩 박스를 예측하는 것을 의미합니다. 먼저 엔드포인트 배포를 위한 점프스타트 아티팩트를 검색합니다. 하지만 base_predictor 대신 우리가 미세 조정한 `od_estimator'를 배포합니다.

---

In [ ]:
inference_instance_type = instance_types.retrieve_default(
    region=None,
    model_id=train_model_id,
    model_version=train_model_version,
    scope="inference"
)

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id=train_model_id,
    model_version=train_model_version,
    instance_type=inference_instance_type,
)

endpoint_name = name_from_base(f"jumpstart-example-FT-{train_model_id}-")


## Custom model deploy

In [ ]:
!wget -O ./model.tar.gz https://d1u5vrgwqrgnt2.cloudfront.net/model.tar.gz

In [ ]:
workshop_bucket = sess.default_bucket()

In [ ]:
!aws s3 cp ./model.tar.gz s3://{workshop_bucket}

In [ ]:
model_url="s3://"+workshop_bucket+"/model.tar.gz"

In [ ]:
from sagemaker.model import Model

#model_url="s3://sagemaker-us-east-1-417959117910/jumpstart-example-sd-training/output/jumpstart-example-mo-230426-2130-001-4d522ff5/output/model.tar.gz"

model = Model(image_uri=deploy_image_uri, 
              model_data=model_url, 
              role=aws_role)

In [ ]:
from datetime import datetime

endpoint_name = f"SD-DEMO-{datetime.utcnow():%Y-%m-%d-%H%M}"
print("EndpointName =", endpoint_name)

In [ ]:
initial_instance_count=1
# initial_instance_count=1 # Example

instance_type=inference_instance_type
# instance_type='ml.m4.xlarge' # Example

model.deploy(
    initial_instance_count=initial_instance_count,
    instance_type=instance_type,
    endpoint_name=endpoint_name
)

In [ ]:
from sagemaker.predictor import Predictor

pred = Predictor(endpoint_name)




다음으로 미세 조정된 모델을 쿼리하고, 응답을 파싱하고, 생성된 이미지를 표시합니다. 이러한 함수는 섹션 [2.3. Query endpoint and parse response](#2.3.-Query-endpoint-and-parse-response)에 구현되어 있습니다. 해당 셀을 실행해 주세요.

In [ ]:
text = "a photo of a Doppler dog with a hat"
query_response = query(pred, text)
img, prmpt = parse_response(query_response)
display_img_and_prompt(img, prmpt)



[2.4. Supported Inference parameters](#2.4.-Supported-Inference-parameters) 는 미세 조정 모델에서도 지원됩니다. [2.5. Compressed Image Output](#2.5.-Compressed-Image-Output) 에서와 같이 `accept`을 변경하여 압축된 이미지 출력을 받을 수도 있습니다.

---
Next, we delete the endpoint corresponding to the finetuned model.

---

In [ ]:
import boto3
# Create a low-level SageMaker service client.
sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

# Delete endpoint
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

### 4. Conclusion
---

이 튜토리얼에서는 JumpStart를 사용하여 사전 학습된 Stable Diffusion 모델을 SageMaker에 배포하는 방법을 배웠습니다. 우리는 Stable Diffusion 모델이 텍스트에서 매우 사실적인 이미지를 생성할 수 있다는 것을 알았습니다. JumpStart는 Stable Diffusion 1과 Stable Diffusion 2와 FP16 버전을 모두 제공합니다. 또한 JumpStart는 다양한 테마와 언어로 이미지를 생성하도록 훈련된 84개의 확산 모델을 추가로 제공합니다. 코드를 직접 작성하지 않고도 이러한 모델을 배포할 수 있습니다. 특정 모델을 배포하려면 [2.1. Select a Model](#2.1.-Select-a-Model) 에서  `model_id` 를 선택하면 됩니다. 

추론 중에 적절한 파라미터를 선택하여 이미지 생성 프로세스를 조정할 수 있습니다. 이러한 파라미터를 설정하는 방법에 대한 지침은 [2.4. Supported Inference parameters](#2.4.-Supported-Inference-parameters)에 나와 있습니다. 또한 대용량 이미지 페이로드를 반환하면 응답 크기 제한 문제가 발생할 수 있다는 사실도 확인했습니다. JumpStart는 엔드포인트에서 이미지를 인코딩하고 표시하기 전에 노트북에서 디코딩하여 이미지를 처리합니다. 마지막으로 고품질 이미지를 생성하는 데 있어 신속한 엔지니어링이 얼마나 중요한 단계인지 살펴보았습니다. 직접 프롬프트를 설정하는 방법을 설명하고 좋은 프롬프트의 몇 가지 예를 살펴보았습니다.

사전 학습된 Stable Diffusion 모델에 대한 추론에 대해 자세히 알아보려면 블로그 [Generate images from text with the stable diffusion model on Amazon SageMaker JumpStart](https://aws.amazon.com/blogs/machine-learning/generate-images-from-text-with-the-stable-diffusion-model-on-amazon-sagemaker-jumpstart/) 을 참조하십시오.

인상적인 이미지를 만드는 것이 예술에서 NFT에 이르기까지 다양한 산업에서 사용될 수 있지만, 오늘날 우리는 AI를 개인화할 수도 있을 것으로 기대합니다. JumpStart는 사전 학습된 모델에 미세 조정 기능을 제공하므로 최소 5개의 학습 이미지로 모델을 자신의 사용 사례에 맞게 조정할 수 있습니다. 이는 아트, 로고, 맞춤형 디자인, NFT 등을 만들거나 애완 동물이나 아바타의 맞춤형 AI 이미지를 생성하는 것과 같은 재미있는 것을 만들 때 유용할 수 있습니다. Stable Diffusion 미세 조정에 대해 자세히 알아보려면 블로그 [Fine-tune text-to-image Stable Diffusion models with Amazon SageMaker JumpStart](https://aws.amazon.com/blogs/machine-learning/fine-tune-text-to-image-stable-diffusion-models-with-amazon-sagemaker-jumpstart/) 을 참조하십시오.